# パラメータの初期値を変えながら、2層ニューラルネットワークで単純な回帰問題を解く

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from common.loss import mean_squared_error
from collections import OrderedDict
from common.layers import Affine, ReLU,MeanSquaredLoss, LeakyReLU
from common.optimizer import RMSProp, SGD

## 2層ニューラルネットワーククラス

In [ ]:
class TwoLayerNet:

    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
        # 重みの初期化
        self.params = {}
        np.random.seed(1111)
        
        # (1)パラメータの初期値を全て0にした場合
        self.params["W1"] = np.zeros((input_size, hidden_size))
        self.params["b1"] = np.zeros(hidden_size) 
        self.params["W2"] = np.zeros((hidden_size, output_size))
        self.params["b2"] = np.zeros(output_size) 

        
        # レイヤの生成
        self.layers = OrderedDict() # 順番付きdict形式.
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] =  ReLU()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])
        self.lastLayer = MeanSquaredLoss() # 出力層
        
    def predict(self, x):
        """
        予測関数
        x : 入力
        """
        for layer in self.layers.values():
            # 入力されたxを更新していく = 順伝播計算
            x = layer.forward(x)
        
        return x
        
    def loss(self, x, t):
        """
        損失関数
        x:入力データ, t:教師データ
        """
        y = self.predict(x)
        return self.lastLayer.forward(y, t)
    
    def accuracy(self, x, t):
        """
        精度
        """
        y = self.predict(x)
        return mean_squared_error(y, t)

        
    def gradient(self, x, t):
        """
        全パラメータの勾配を計算
        """
        
        # 順伝播
        self.loss(x, t)

        # 逆伝播
        dout = self.lastLayer.backward(dout=1) # 出力層
        
        ## doutを逆向きに伝える 
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # dW, dbをgradsにまとめる
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W2'], grads['b2'] = self.layers['Affine2'].dW, self.layers['Affine2'].db    

        return grads
    

## データの生成

In [ ]:
a = 3
b = 0.5
c = -0.1
x = np.arange(-0.5, 0.5, 0.01)
error = np.random.rand(x.size) * 0.1
y = a * x**3 + b * x**2 + c + error
plt.scatter(x, y)
plt.xlabel("x")
plt.ylabel("y")
plt.show()

## ミニバッチ学習

In [ ]:
x = x.reshape(-1, 1)
t = y.reshape(-1, 1)

ecpochs = 5000
batch_size = 25
lr = 0.01

# 繰り返し回数
xsize = x.shape[0]
iter_num = np.ceil(xsize / batch_size).astype(np.int)

# 2層NNのオブジェクト生成
tnet = TwoLayerNet(input_size=1, hidden_size=10, output_size=1, weight_init_std=0.01)

# 最適化手法
optimizer = RMSProp(lr=lr, rho=0.9)
    
# 初期パラメータの確認
print("初期パラメータ")
print(tnet.params["W1"].round(4))
print(tnet.params["b1"].round(4))
print(tnet.params["W2"].round(4))
print(tnet.params["b2"].round(4))


li_loss = []
for epoch in range(ecpochs):
    
    # シャッフル
    idx = np.arange(xsize)
    np.random.shuffle(idx)

    for it in range(iter_num):
        """
        ランダムなミニバッチを順番に取り出す
        """
        mask = idx[batch_size*it : batch_size*(it+1)]
    
        # ミニバッチの生成
        x_train = x[mask]
        t_train = t[mask]
        
        # 勾配の計算 (誤差逆伝播法を用いる) 
        grads = tnet.gradient(x_train, t_train)

        # パラメータの更新
        optimizer.update(tnet.params, grads)

    # 学習経過の記録
    loss = tnet.loss(x, t)
    li_loss.append(loss)

### 学習結果の確認

In [ ]:
# lossのグラフ化
pd.DataFrame(li_loss).plot(legend=False)
plt.ylabel("loss")
plt.xlabel("epochs")
plt.show()

In [ ]:
# 訓練精度の確認
y_pred = tnet.predict(x)
print("mse=",mean_squared_error(y_pred, t))

# 学習データと予測データの比較1
plt.figure(figsize=(4.3,4))
plt.scatter(y, y_pred)
plt.xlabel("y")
plt.ylabel("y_pred")
plt.xlim([-0.4,0.5])
plt.ylim([-0.4,0.5])
plt.show()

# 学習データと予測データの比較2
plt.scatter(x, y)
plt.xlabel("x")
plt.ylabel("y")
plt.scatter(x, y_pred, c="r")
plt.xlabel("x")
plt.ylabel("y_pred")
plt.legend(["y","y_pred"])
plt.ylim([-0.4,0.5])
plt.show()

### パラメータの分布の確認

In [ ]:
# 最終パラメータの確認
print(tnet.params["W1"].round(4))
print(tnet.params["b1"].round(4))
print(tnet.params["W2"].round(4))
print(tnet.params["b2"].round(4))

In [ ]:
# パラメータ分布
for key, value in tnet.params.items():
    plt.hist(value.flatten(), bins=10)
    plt.title(key)
    plt.xlim([-3, 3])
    plt.ylabel("Frequency")
    plt.xlabel("Value of parameter")
    plt.show()

### [演習]
* パラメータの初期値を以下のように設定し、結果を比較してみましょう
#### (1)パラメータの初期値を全て0にした場合
#### (2)Wの初期値を0にし、bの初期値を1にした場合
#### (3)パラメータの初期値を全て1にした場合
#### (4)パラメータの初期値を全て1にし、1つだけ1以外にした場合
#### (5)パラメータの初期値を正規分布にしたがってランダムにサンプリングした場合
#### (6)Wの初期値を正規分布にしたがってランダムにサンプリングし、bの初期値を0にした場合
